In [ ]:
# import pandas as pd
# import numpy as np
# import nltk
# import os
# import json
# import requests 
# import time
# import re
# # import env_miatta as e
# # from wrangle import tokenize,clean,nlp_wrangle,intersection_list,extra_clean_column as w
# import matplotlib.pyplot as plt
# import unicodedata
# from bs4 import BeautifulSoup
# from typing import Dict, List, Optional, Union, cast
# from env_miatta import github_token, github_username
# from pprint import pprint
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.ensemble import RandomForestClassifier
# from nltk.tokenize.toktok import ToktokTokenizer
# from nltk.corpus import stopwords
# %matplotlib inline
# from time import strftime
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import numpy as np
import time

# Data visualization and manipulation
import matplotlib.pyplot as plt
import wordcloud 
import seaborn as sns
from pprint import pprint

# Natural language processing and modeling
import nltk.sentiment
import nltk
import re
from scipy.stats import f_oneway, stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import ToktokTokenizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

# SQL credentials and data acquisition
# import env as e
# import acquire as a
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from bs4 import BeautifulSoup

# GitHub API credentials
# from env import github_token, github_username

import warnings

# ACQUIRE

In [ ]:
# import json
# import pandas as pd

# def combine_json_files_and_save(json_file_names, output_file_name):
#     combined_data = []

#     for file_name in json_file_names:
#         with open(file_name, "r") as json_file:
#             data = json.load(json_file)
#             combined_data.extend(data)

#     with open(output_file_name, "w") as combined_json_file:
#         json.dump(combined_data, combined_json_file, indent=1)

# # List of JSON file names
# json_file_names = [
#     'data1.json',
#     'data3.json',
#     'data4.json',
#     # ... (other file names)
#     'data24.json'
# ]

# # Output file name for combined data
# output_file_name = "data2.json"

# # Call the function to combine JSON files and write the output
# combine_json_files_and_save(json_file_names, output_file_name)

# # Read the combined JSON data into a DataFrame
# words_df = pd.read_json(output_file_name)


DATA TYPE SUMMARY

26 integer data types originally now 3
9 object data type originally now 1
0 null values
no missing values

In [ ]:
# Aquire data using methods described above.
words_df = pd.read_json('data2.json')
words_df



In [ ]:
# Dataset columns
words_df.columns

In [ ]:
words_df.language.value_counts()

In [ ]:
# To inspect the first few rows of the DataFrame, you can use the head functiotelco.head()
words_df.head().T


In [ ]:
# Check for total of duplicates in data set 
words_df.duplicated().sum()

In [ ]:
# column datatypes and missign values
words_df.info()

In [ ]:
# observed numerical values
words_df.describe()


In [ ]:
words_df.shape


In [ ]:
# Check for missing values
missing_values = words_df.isnull()
missing_values


In [ ]:
words_df.isnull().sum()


In [ ]:
# Drop rows with any missing values
words_df_dropped = words_df.dropna()
words_df_dropped


In [ ]:
rows_with_missing_language = words_df[words_df['language'].isnull()]

# Change data types of the isolated rows to 'object'
rows_with_missing_language = rows_with_missing_language.astype({'language': 'object'})

# Display the modified isolated rows
rows_with_missing_language


In [ ]:
# column datatypes 
words_df.info()

In [ ]:
# Replace null values in the 'language' column with "Other"
words_df['language'].fillna("Other", inplace=True)

In [ ]:
#column datatypes 
words_df.info()

In [ ]:
words_df

# PREPARE

### Data Cleaning:
    
- Drop unnecessary axis
- Rename
- Find nulls
- Drop nulls
- Check preperation
- The data set has 4 columns and 1,470 rows
- Each row represents individual employee numerical data
- Each column is attributes of the employees

In [ ]:

def tokenize(text):
    """
    Tokenizes the words in the input string.
    """
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

def nlp_wrangle():
    """
    Performs data wrangling for natural language processing (NLP) tasks.
    Returns a processed DataFrame for NLP analysis.
    """
    # Load data from JSON file
    df = pd.read_json('data2.json')
    
    # Tokenize and clean contents
    df['clean_contents'] = df.readme_contents.apply(tokenize).apply(' '.join)
    df['clean_contents'] = df.clean_contents.apply(clean).apply(' '.join)
    
     # Words to remove
    words_to_remove = ['http', 'com', '124', 'www','github', 'top', 'go','107', '0','1','2','3','4', '5', '6', '7', '8','9', 'md','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'em', 'abbr', 'q','ins', 'del', 'dfn', 'kbd', 'pre', 'samp', 'var', 'br', 'div', 'a', 'img', 'param', 'ul','ol', 'li', 'dl', 'dt', 'dd']

    # Remove specific words from clean_contents
    for word in words_to_remove:
        df['clean_contents'] = df['clean_contents'].str.replace(word, '')

    # Add message_length and word_count columns
    df['message_length'] = df['clean_contents'].str.len()
    df['word_count'] = df.clean_contents.apply(clean).apply(len)

    # Keep only top languages and assign others to 'Other'
    languages_to_keep = ['JavaScript', 'Python', 'Java', 'TypeScript', 'HTML']
    df['language'] = np.where(df['language'].isin(languages_to_keep), df['language'], 'Other')

    # Filter DataFrame based on conditions
    df = df.loc[(df['word_count'] <= 10000) & (df['message_length'] <= 60000)]

    return df

def intersection_list():
    words_df = nlp_wrangle()
    readme_words_list = words_df.clean_contents.to_list()
    readme_words_list

    readme_words = []
    for list in readme_words_list:
        split_list = list.split()
        readme_words.append(split_list)

    words_list = []
    for _ in readme_words:
        for el in _:
            words_list.append(el)

    dictionary_words = pd.read_csv('/usr/share/dict/words', header=None)
    dictionary_words = dictionary_words.drop(index=[122337,122338])
    dictionary_words = dictionary_words.squeeze()
    intersect = set(words_list) & set(dictionary_words)
    intersect = sorted(intersect)
    return intersect

def extra_clean_column(words_df):
    extra_clean_article = []
    for i in words_df.index:
        article_words = words_df.clean_contents[i].split()
        extra_clean = set(intersect) & set(article_words)
        extra_clean = sorted(extra_clean)
        extra_clean = ' '.join(extra_clean)
        extra_clean_article.append(extra_clean)

    words_df = words_df.assign(extra_clean_contents = extra_clean_article) 
    return words_df

In [ ]:
words_df = nlp_wrangle()
intersect = intersection_list()
words_df = extra_clean_column(words_df)
words_df

### Question 2: Does the presence of specific libraries in the README file correspond with the programming language used?

### Question 4: What are the least frequently used words throughout the dataset and for each language?


### TRAIN SPLIT

# EXPLORE

In [ ]:
a= set(range(0,118)) - set(list(words_df_w.index))
# a = list(a)
a

In [ ]:
words_df_temp = words_df[words_df.index == 41]

In [ ]:
words_df_temp.readme_contents

In [ ]:
cleaned_list = []

def clean(text: str) -> list: 
    """
    Cleans up the input text data.
    """
    text = (text.encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
    
    words = re.sub(r'[^\w\s]', ' ', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

for content in words_df.readme_contents:
    cleaned_text = clean(content)
    cleaned_list.append(cleaned_text)
c

In [ ]:
def split_data(df, variable):
    '''
    take in a DataFrame and target variable. return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    train_validate, test = train_test_split(df, test_size=.20, random_state=123, stratify=df[variable])
    
    train, validate = train_test_split(train_validate, 
                                       test_size=.25, 
                                       random_state=123,
                                      stratify = train_validate[variable])
    return train, validate, test

In [ ]:
cleaned_list


In [ ]:
se = pd.Series(cleaned_list)
words_df['cleaned_text'] = se.values


In [ ]:
words_df


In [ ]:
train, validate, test = split_data(df, 'language')
train.head()

In [ ]:
    separate specific words by language category

In [ ]:
# we can do that process with a join on a Series and not just a list
# we will do that all words and categories
# we will pass our basic cleaning on top of that

JavaScript_words = clean(' '.join(train[train.language=='JavaScript']['clean_contents']))
Python_words = clean(' '.join(train[train.language=='Python']['clean_contents']))
Java_words = clean(' '.join(train[train.language=='Java']['clean_contents']))
TypeScript_words = clean(' '.join(train[train.language=='TypeScript']['clean_contents']))
HTML_words = clean(' '.join(train[train.language=='HTML']['clean_contents']))
Other_words = clean(' '.join(train[train.language=='Other']['clean_contents']))
all_words = clean(' '.join(train['clean_contents']))

In [ ]:
JavaScript_words      

In [ ]:
Python_words              

In [ ]:
Java_words               

In [ ]:
HTML_words

In [ ]:
C_words

In [ ]:
Jupyter Notebook_words

In [ ]:
Shell_words

In [ ]:
Ruby_words
C++_words
CSS_words                  
Go_words
TypeScript           3
SCSS                 2
PowerShell           1
Kotlin               1
Bicep                1
HCL                  1
Vim script           1
Objective-C          1
Vue        